I fixed the TPCF issues, but now the delta sigma projection integration is being troublesome. I'm gonna do them for a few rp_bins to figure out what is sensible agian. 

In [2]:
import numpy as np
from glob import glob
from os import path

ImportError: cannot import name _distributor_init

In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import matplotlib.colors as colors

AttributeError: 'tuple' object has no attribute 'type'

In [ ]:
from pearce.mocks.kittens import DarkSky
from pearce.mocks import tpcf as pearce_tpcf
from halotools.mock_observables import tpcf as halotools_tpcf
from halotools.empirical_models import Zheng07Cens, Zheng07Sats
from collections import OrderedDict
from time import time
from scipy.optimize import minimize_scalar
import yaml

In [ ]:
output_dir = './'# '/home/users/swmclau2/Git/pearce/bin/covmat/ds14_covmat/'

In [ ]:
config_fname = 'xi_cosmo_trainer.yaml'

with open(path.join(output_dir, config_fname), 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

nd = float(cfg['HOD']['fixed_nd'] )
min_ptcl = int(cfg['HOD']['min_ptcl'])
r_bins = np.array(cfg['observation']['bins'] ).astype(float)

hod_param_ranges =  cfg['HOD']['ordered_params']


logMmin_bounds = hod_param_ranges['logMmin']


del hod_param_ranges['logMmin']

In [ ]:
def make_LHC(ordered_params, N, seed = None):

    if seed is None:
        seed = int(time())
    np.random.seed(seed)

    points = []
    # by linspacing each parameter and shuffling, I ensure there is only one point in each row, in each dimension.
    for plow, phigh in ordered_params.itervalues():
        point = np.linspace(plow, phigh, num=N)
        np.random.shuffle(point)  # makes the cube random.
        points.append(point)
    return np.stack(points).T


def add_logMmin(hod_params, cat):

    hod_params['logMmin'] = 13.0 #initial guess
    #cat.populate(hod_params) #may be overkill, but will ensure params are written everywhere
    def func(logMmin, hod_params):
        hod_params.update({'logMmin':logMmin})
        return (cat.calc_analytic_nd(hod_params, min_ptcl = min_ptcl) - nd)**2

    res = minimize_scalar(func, bounds = logMmin_bounds, args = (hod_params,), options = {'maxiter':100}, method = 'Bounded')

    # assuming this doens't fail
    #print 'logMmin', res.x
    hod_params['logMmin'] = res.x

In [ ]:
from pearce.mocks.kittens import TestBox
cat = TestBox(boxno = 0, realization = 0, system = 'sherlock')
cat.load(1.0, HOD='zheng07', particles = True, downsample_factor = 1e-2)

In [ ]:
# TODO seed here for constant HODs
# TODO maybe just do 5, 10 may be overkill
N = 10
LHC = make_LHC(hod_param_ranges, N, 24)
hod_dicts = [dict(zip(hod_param_ranges.keys(), vals)) for vals in LHC]

In [ ]:
cat.populate(hod_dicts[0])

In [ ]:
cat._get_cosmo_param_names_vals()

In [ ]:
from halotools.mock_observables import return_xyz_formatted_array, delta_sigma, tpcf

In [ ]:
def calc_ds(cat, bins):
    #n_cores = self._check_cores(n_cores)
    n_cores = 2
    
    x_g, y_g, z_g = [cat.model.mock.galaxy_table[c] for c in ['x', 'y', 'z']]
    pos_g = return_xyz_formatted_array(x_g, y_g, z_g, period=cat.Lbox)

    x_m, y_m, z_m = [cat.halocat.ptcl_table[c] for c in ['x', 'y', 'z']]
    pos_m = return_xyz_formatted_array(x_m, y_m, z_m, period=cat.Lbox)

    rp_bins = bins #if not angular else self._rp_from_ang(bins)

    # Halotools wnats downsampling factor defined oppositley
    # TODO verify little h!
    # TODO maybe split into a few lines for clarity
    return delta_sigma(pos_g / cat.h, pos_m / cat.h, cat.pmass / cat.h,
                   downsampling_factor=1. / cat._downsample_factor, rp_bins=rp_bins,
                   period=[10*cat.Lbox/cat.h, 10*cat.Lbox/cat.h, cat.Lbox / cat.h], num_threads=n_cores, cosmology=cat.cosmology)[1] / (1e12)


In [ ]:
from scipy.interpolate import interp1d
import pyccl as ccl
from scipy.integrate import quad

In [ ]:
from halotools.mock_observables import delta_sigma_from_precomputed_pairs, total_mass_enclosed_per_cylinder

In [ ]:
def calc_ds2(cat, bins):
    #n_cores = self._check_cores(n_cores)
    n_cores = 2
    
    x_g, y_g, z_g = [cat.model.mock.galaxy_table[c] for c in ['x', 'y', 'z']]
    pos_g = return_xyz_formatted_array(x_g, y_g, z_g, period=cat.Lbox)

    x_m, y_m, z_m = [cat.halocat.ptcl_table[c] for c in ['x', 'y', 'z']]
    pos_m = return_xyz_formatted_array(x_m, y_m, z_m, period=cat.Lbox)

    rp_bins = bins #if not angular else self._rp_from_ang(bins)

    tm = total_mass_enclosed_per_cylinder(pos_g/cat.h, pos_m/cat.h, cat.pmass/cat.h,\
                                                   1./cat._downsample_factor, rp_bins, [None, None, cat.Lbox/cat.h]\
                                         num_threads = n_cores)
    (cens, mass_encl_cens, rp_bins, period, cosmology=halocat.cosmology)
    return delta_sigma_from_precomputed_pairs(pos_g / cat.h, tm, rp_bins, cat.Lbox/cat.h, cat.pmass / cat.h,
                   num_threads=n_cores, cosmology=cat.cosmology)[1] / (1e12)


In [ ]:
rp_bins = np.logspace(-1.0, 1.6, 19)

In [ ]:
names, vals = cat._get_cosmo_param_names_vals()
param_dict = {n: v for n, v in zip(names, vals)}

new_param_dict = {}
new_param_dict['Omega_c'] = param_dict['omch2']/cat.h**2
new_param_dict['Omega_b'] = param_dict['ombh2']/cat.h**2
new_param_dict['n_s'] = param_dict['ns']
new_param_dict['h'] = cat.h
new_param_dict['A_s'] = np.exp(param_dict['ln10As'])/(np.power(10, 10))

In [ ]:
param_dict

In [ ]:
new_param_dict

In [ ]:
rp_bins

In [ ]:
ds = calc_ds(cat, rp_bins)

In [ ]:
ds2 = calc_ds2(cat, rp_bins)

In [ ]:
rpoints = (rp_bins[1:] + rp_bins[:-1])/2.0
plt.plot(rpoints, ds.squeeze(), label = 'Halotools')
plt.plot(rpoints, ds2.squeeze(), label = 'Halotools')
#plt.plot(rpoints, ds_analytic.squeeze(), label = 'Analytic')

plt.legend(loc='best')
plt.loglog();